In [1]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import tensorflow as tf
from tensorflow.keras import *
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import json

In [4]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://10.88.80.42:8470', zone='europe-west4-a')
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)

tpu_strategy = tf.distribute.experimental.TPUStrategy(resolver)

INFO:tensorflow:Initializing the TPU system: 10.88.80.42:8470


INFO:tensorflow:Initializing the TPU system: 10.88.80.42:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [5]:
def identity_block(input_tensor, kernel_size, filters, stage, block):
    filters1, filters2, filters3 = filters
    bn_axis = 3

    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    x = Conv2D(filters1, (1,1), name=conv_name_base+'2a')(input_tensor)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base+'2a')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters2, kernel_size,
               padding='same', name=conv_name_base+'2b')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base+'2b')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters3, (1,1), name=conv_name_base+'2c')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base+'2c')(x)

    x = layers.add([x, input_tensor])
    x = Activation('relu')(x)

    return x

def conv_block(input_tensor, kernel_size, filters, stage, block, strides = (2, 2)):
    filters1, filters2, filters3 = filters
    bn_axis = 3

    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    x = Conv2D(filters1, (1,1), strides=strides,
               name=conv_name_base+'2a')(input_tensor)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base+'2a')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters2, kernel_size, padding='same',
               name=conv_name_base+'2b')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base+'2b')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters3, (1,1), name=conv_name_base+'2c')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base+'2c')(x)

    shortcut = Conv2D(filters3, (1,1), strides=strides,
                      name=conv_name_base+'1')(input_tensor)
    shortcut = BatchNormalization(axis=bn_axis, name=bn_name_base+'1')(shortcut)

    x = layers.add([x, shortcut])
    x = Activation('relu')(x)

    return x

In [6]:
def resnet_50(input_tensor=None, input_shape=None, pooling=None, classes=1000):
    img_input = Input(shape=input_shape)
    bn_axis = 3

    x = Conv2D(64, (7,7), strides=(2,2), padding='same', name='conv1')(img_input)
    x = BatchNormalization(axis=bn_axis, name='bn_conv1')(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((3,3), strides=(2,2), padding='same')(x)

    x = conv_block(x, 3, [64,64,256], stage=2, block='a', strides=(1,1))
    x = identity_block(x, 3, [64,64,256], stage=2, block='b')
    x = identity_block(x, 3, [64,64,256], stage=2, block='c')

    x = conv_block(x, 3, [128,128,512], stage=3, block='a')
    x = identity_block(x, 3, [128,128,512], stage=3, block='b')
    x = identity_block(x, 3, [128,128,512], stage=3, block='c')
    x = identity_block(x, 3, [128,128,512], stage=3, block='d')

    x = conv_block(x, 3, [256,256,1024], stage=4, block='a')
    x = identity_block(x, 3, [256,256,1024], stage=4, block='b')
    x = identity_block(x, 3, [256,256,1024], stage=4, block='c')
    x = identity_block(x, 3, [256,256,1024], stage=4, block='d')
    x = identity_block(x, 3, [256,256,1024], stage=4, block='e')
    x = identity_block(x, 3, [256,256,1024], stage=4, block='f')

    x = conv_block(x, 3, [512,512,2048], stage=5, block='a')
    x = identity_block(x, 3, [512,512,2048], stage=5, block='b')
    x = identity_block(x, 3, [512,512,2048], stage=5, block='c')
    
    x = layers.GlobalMaxPooling2D()(x)

    model = Model(img_input, x, name='resnet50')
    resnet50_weights_path = 'resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'
    model.load_weights(resnet50_weights_path)

    return model

In [7]:
image_generator = ImageDataGenerator(rescale=1./255,
                                     zoom_range=0.2,
                                     rotation_range=45,
                                     horizontal_flip=True,
                                     validation_split=0.05)

In [8]:
INPUT_SHAPE = (224, 224, 3)
BATCH_SIZE = 1024

In [9]:
def train_generator():
    training_set = image_generator.flow_from_directory('../Dataset/train/herbarium/images/',
                                                       target_size=INPUT_SHAPE[:2],
                                                       batch_size=BATCH_SIZE,
                                                       class_mode='categorical',
                                                       subset='training')
    return training_set

def validation_generator():
    validation_set = image_generator.flow_from_directory('../Dataset/train/herbarium/images/',
                                                         target_size=INPUT_SHAPE[:2],
                                                         batch_size=BATCH_SIZE,
                                                         class_mode='categorical',
                                                         subset='validation')
    return validation_set

In [14]:
train_data = train_generator()
validation_data = validation_generator()

Found 305208 images belonging to 998 classes.
Found 15552 images belonging to 998 classes.


In [11]:
CLASSES = 998
EPOCHS = 4

In [12]:
with tpu_strategy.scope():
    resnet_base = resnet_50(input_shape=INPUT_SHAPE)    
    for layer in resnet_base.layers:
        layer.trainable = True
        
    resnet_output = resnet_base.layers[-1].output
    top = Flatten()(resnet_output)
    top = Dense(units=2048, activation='relu')(top)
    top = Dense(units=2048, activation='relu')(top)
    top = Dense(units=CLASSES, activation='softmax')(top)
    
    model = Model(resnet_base.input, top)    
    model.summary()
    model.compile(optimizer=Adam(learning_rate=1.1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        input_1[0][0]                    
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation (Activation)         (None, 112, 112, 64) 0           bn_conv1[0][0]                   
______________________________________________________________________________________________

In [18]:
model.fit(train_data,
          steps_per_epoch=train_data.samples//BATCH_SIZE,
          validation_data=validation_data,
          validation_steps=validation_data.samples//BATCH_SIZE,
          epochs=EPOCHS)

  ...
    to  
  ['...']


  ...
    to  
  ['...']


  ...
    to  
  ['...']


  ...
    to  
  ['...']


Train for 298 steps, validate for 15 steps
Epoch 1/4
  1/298 [..............................] - ETA: 7:18:23

NotFoundError: {{function_node __inference_distributed_function_76431}} No registered 'PyFunc' OpKernel for 'CPU' devices compatible with node {{node PyFunc}}
	.  Registered:  <no registered kernels>

	 [[PyFunc]]
	 [[MultiDeviceIteratorGetNextFromShard]]
	 [[RemoteCall]]
	 [[IteratorGetNextAsOptional]]

In [ ]:
model.save('../Models/resnet50_retrain.h5')

In [ ]:
with open('../Mappings/class_mappings_resnet_50_retrain.json', 'w', encoding='utf-8') as f:
    json.dump(validation_set.class_indices, f, ensure_ascii=False, indent=4)